In [26]:
import pandas as pd
import numpy as np
import os
cwd = os.getcwd()

ROOT = os.path.join(cwd, '../')
DATA_FOLDER = os.path.join(ROOT, 'data/')

In [27]:
df = pd.read_pickle(os.path.join(DATA_FOLDER, 'saeb_treated.pkl'))
df = df.dropna()
df = df[df['desempenho_aluno'] != 'Avançado']
print(len(df))
df.head()

48910


,ano,sigla_uf,escola_publica,id_aluno,situacao_censo,disciplina,proficiencia,erro_padrao,proficiencia_saeb,erro_padrao_saeb,desempenho_aluno,sexo,raca_cor,ano_nascimento,idade,escolaridade_mae,mae_sabe_ler_escrever,mae_le,escolaridade_pai,pai_sabe_ler_escrever,pai_le,responsaveis_incentivam_realizacao_licao_casa,responsaveis_incentivam_estudos,responsaveis_incentivam_leitura,responsaveis_incentivam_comparecer_aulas,possui_computador,tempo_lazer,leitura_jornais,leitura_livros_geral,leitura_literatura_infantojuvenil,leitura_historia_quadrinhos,leitura_revista_comportamento,leitura_revistas_geral,leitura_internet,frequenta_biblioteca,frequenta_cinema,frequenta_espetaculo_exposicao,frequenta_festas_comunidade,gosta_estudar_disciplina,faz_licao_casa,correcao_licao_casa,utiliza_biblioteca_escola,rede,id_escola,serie,turno,desempenho_aluno_cat
27111,2015,PB,1,17523018,0,MT,-0.627672,0.648351,214.882332,36.237847,Insuficiente,A,A,1997,18,4.0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2,25083490,9,3,0
27112,2015,PB,1,17491737,0,LP,-1.027257,0.354996,193.389947,19.557923,Insuficiente,B,B,1997,18,1.0,1,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,4.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2,61134907,9,1,0
27115,2015,PB,1,17491737,0,MT,-0.807123,0.639382,204.852398,35.736548,Insuficiente,B,B,1997,18,1.0,1,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,4.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2,61134907,9,1,0
27122,2015,PB,1,17488937,0,LP,-0.341425,0.380753,231.174739,20.976963,Básico,B,D,1997,18,0.0,0,0.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0,2,25027573,9,1,1
27123,2015,PB,1,17488937,0,MT,-0.906444,0.545208,199.301116,30.472944,Insuficiente,B,D,1997,18,0.0,0,0.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0,2,25027573,9,1,0


In [28]:
exclude = ['ano', 'sigla_uf', 'id_aluno', 'situacao_censo', 'disciplina', 'proficiencia', 'erro_padrao', 'proficiencia_saeb',
       'erro_padrao_saeb', 'sexo', 'raca_cor','ano_nascimento', 'rede', 'id_escola', 'serie', 'turno', 'escola_publica', 'desempenho_aluno']

In [29]:
# print unique values for each column

for col in df.columns:
    if col not in exclude:
        print(col, df[col].unique())

idade [18 17 16 15 14 13]
escolaridade_mae [4. 1. 0. 2. 3. 5.]
mae_sabe_ler_escrever [1 0]
mae_le [1. 0.]
escolaridade_pai [1. 3. 4. 2. 0. 5.]
pai_sabe_ler_escrever [1. 0.]
pai_le [1. 0.]
responsaveis_incentivam_realizacao_licao_casa [1. 0.]
responsaveis_incentivam_estudos [1. 0.]
responsaveis_incentivam_leitura [1. 0.]
responsaveis_incentivam_comparecer_aulas [1. 0.]
possui_computador [1. 0. 2. 3. 4.]
tempo_lazer [1. 4. 2. 3. 0.]
leitura_jornais [1. 0. 2.]
leitura_livros_geral [1. 2. 0.]
leitura_literatura_infantojuvenil [1. 2. 0.]
leitura_historia_quadrinhos [1. 2. 0.]
leitura_revista_comportamento [1. 2. 0.]
leitura_revistas_geral [1. 2. 0.]
leitura_internet [0. 1. 2.]
frequenta_biblioteca [1. 0. 2.]
frequenta_cinema [0. 2. 1.]
frequenta_espetaculo_exposicao [1. 2. 0.]
frequenta_festas_comunidade [1. 2. 0.]
gosta_estudar_disciplina [0. 1.]
faz_licao_casa [1. 2. 0.]
correcao_licao_casa [1. 2. 0.]
utiliza_biblioteca_escola [1. 2. 0.]
desempenho_aluno_cat [0 1 2]


In [30]:
X = df.drop(columns=exclude + ['desempenho_aluno_cat'])
y = df['desempenho_aluno_cat']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [20]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
X, y = rus.fit_resample(X, y)

In [31]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
y_train.value_counts()

desempenho_aluno_cat
1    21775
0    21725
2    21650
Name: count, dtype: int64

In [34]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.60      0.59      5421
           1       0.50      0.34      0.41      5371
           2       0.59      0.73      0.65      5496

    accuracy                           0.56     16288
   macro avg       0.55      0.56      0.55     16288
weighted avg       0.55      0.56      0.55     16288

[[3278 1104 1039]
 [1760 1852 1759]
 [ 689  773 4034]]
0.56262278978389


In [ ]:
# training the model on training set 
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]}
grid = GridSearchCV(estimator=SVC(kernel='rbf', decision_function_shape='ovo'), param_grid=param_grid, cv=5)

%time grid.fit(X_train, y_train)
print(grid.best_params_)
